Подход 1: градиентный бустинг "в лоб"

In [1]:
import pandas
# 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
features = pandas.read_csv('features.csv', index_col='match_id')
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [2]:
#Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующиев тестовой выборке).:
y = features['radiant_win'].to_frame()
#Итог матча: duration: длительность; radiant_win: 1, если победила команда Radiant, 0 — иначе; 
#Состояние башен и барраков к концу матча: tower_status_radiant; tower_status_dire;barracks_status_radiant; barracks_status_dire
features.drop(['duration',
               'radiant_win',
               'tower_status_radiant', 
               'tower_status_dire', 
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1, inplace=True)
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [3]:
# 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. 
def check_missing(features):
    list_of_columns=list(features)
    num_rows=len(features)
    print ("N_features %.f, N_rows %.f" % (len(list_of_columns),num_rows))

    list_of_incomplete=[]
    print ("Missing: ")
    for i in range(0,len(list_of_columns)-1):
        i_count = features[list_of_columns[i]].count()
        if i_count!=num_rows:
            print ("%s , %.f " % (list_of_columns[i],100-i_count/num_rows*100))
            list_of_incomplete.append((i,list_of_columns[i],100-i_count/num_rows*100))
#list_of_columns[len(list_of_columns)-1]
    return list_of_incomplete


In [4]:
list_initial = check_missing(features)

N_features 102, N_rows 97230
Missing: 
first_blood_time , 20 
first_blood_team , 20 
first_blood_player1 , 20 
first_blood_player2 , 45 
radiant_bottle_time , 16 
radiant_courier_time , 1 
radiant_flying_courier_time , 28 
radiant_first_ward_time , 2 
dire_bottle_time , 17 
dire_courier_time , 1 
dire_flying_courier_time , 27 


In [5]:
#Много ли пропусков в данных? 
len(list_initial)

list_initial.sort(key=lambda x:x[2])
list_initial

[(95, 'dire_courier_time', 0.6952586650210861),
 (87, 'radiant_courier_time', 0.7117144914121241),
 (93, 'radiant_first_ward_time', 1.8883060783708743),
 (86, 'radiant_bottle_time', 16.138023243854775),
 (94, 'dire_bottle_time', 16.602900339401415),
 (82, 'first_blood_time', 20.110048338990026),
 (83, 'first_blood_team', 20.110048338990026),
 (84, 'first_blood_player1', 20.110048338990026),
 (96, 'dire_flying_courier_time', 26.841509822071373),
 (88, 'radiant_flying_courier_time', 28.261853337447292),
 (85, 'first_blood_player2', 45.240152216394115)]

In [6]:
# 3. Замените пропуски на нули с помощью функции fillna(). 
features_clean = features.fillna(0)
#Check:
list_of_mis_after_clean = check_missing(features_clean)

N_features 102, N_rows 97230
Missing: 


In [7]:
# 4. Какой столбец содержит целевую переменную? Запишите его название.
list(y)


['radiant_win']

In [8]:
# 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг
# над деревьями на имеющейся матрице "объекты-признаки". 
#Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), 
#не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице 
#отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами 
#при оценивании качества. 
from sklearn.cross_validation import KFold,cross_val_score
kf = KFold(y.size, n_folds=5, shuffle=True, random_state=241)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
#Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной 
#кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте 
#следующие значения для количества деревьев: 10, 20, 30). 

#Долго ли настраивались классификаторы? 
import time
import datetime
start_time = datetime.datetime.now()
time.sleep(3) # вместо этой строчки разместить замеряемый код
print ('Time elapsed:', datetime.datetime.now() - start_time)


Time elapsed: 0:00:03.001411


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
gb_qual = []
n_trees = [10, 20, 30, 50, 70,100,150,200]
Y=np.array(y)
for i in n_trees:
    gb = GradientBoostingClassifier(n_estimators=i, random_state=241)
    start_time = datetime.datetime.now()
    gb_scores = cross_val_score(gb, features_clean, Y.ravel(), cv=kf, scoring='roc_auc')
    run_time=datetime.datetime.now() - start_time
    print ('Time elapsed:', run_time)
    print (gb_scores)
    gb_qual.append((i,run_time,np.mean(gb_scores)))

Time elapsed: 0:00:19.296906
[0.66943496 0.65627754 0.66390454 0.66281223 0.66950933]
Time elapsed: 0:00:35.280699
[0.68939011 0.67567277 0.68201026 0.67977584 0.68741889]
Time elapsed: 0:00:56.224454
[0.69641668 0.68365441 0.68731862 0.68728458 0.69280674]
Time elapsed: 0:01:23.578763
[0.70382382 0.69325852 0.69452532 0.69565719 0.7000093 ]
Time elapsed: 0:01:55.002650
[0.70784114 0.69833267 0.6990506  0.70019753 0.70473656]
Time elapsed: 0:02:43.064857
[0.71211028 0.70275727 0.70340522 0.70450889 0.70884943]
Time elapsed: 0:04:06.177800
[0.71546705 0.70731614 0.70793497 0.70887821 0.71252658]
Time elapsed: 0:05:23.668560
[0.71884063 0.71030142 0.71135838 0.71139223 0.71574489]


In [11]:
#Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, 
#скорее всего, продолжит расти при дальнейшем его увеличении?
gb_qual

[(10, datetime.timedelta(0, 19, 296906), 0.6643877206345742),
 (20, datetime.timedelta(0, 35, 280699), 0.6828535735340822),
 (30, datetime.timedelta(0, 56, 224454), 0.6894962060591201),
 (50, datetime.timedelta(0, 83, 578763), 0.6974548316948366),
 (70, datetime.timedelta(0, 115, 2650), 0.7020316980434622),
 (100, datetime.timedelta(0, 163, 64857), 0.7063262181631453),
 (150, datetime.timedelta(0, 246, 177800), 0.7104245908535362),
 (200, datetime.timedelta(0, 323, 668560), 0.713527508807472)]

Report

Q1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
A1.:
1. first_blood_player2 (второй игрок, причастный к событию) - в 55% случаев только 1 игрок причастен к событию;
2. radiant_flying_courier_time (время приобретения предмета "flying_courier") - в 72% случаев этот предмет не покупается

Q2. Как называется столбец, содержащий целевую переменную?
A2.:
['radiant_win']

Q3.1 Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? 
Q3.2 Какое качество при этом получилось?
A3.1: для 30 деревьев 56 секунд, 
A3.2: качество = 0.69

Q4.1 Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? 
Q4.2 Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
A4.1: Нет, после 30 деревьев качество можно увеличить до 71% на 200 деревьях, но лучше стоит подобрать более оптимальный алгоритм
A4.2: Можно обучать на подвыборках, либо уменьшить глубину

In [12]:
gb = GradientBoostingClassifier(n_estimators=250, random_state=241)
gb.fit(features_clean, Y.ravel())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=250,
              presort='auto', random_state=241, subsample=1.0, verbose=0,
              warm_start=False)

In [13]:
features_test_clean = features_test.fillna(0)
gb.predict_proba(features_test_clean)[:,1]

array([0.45677799, 0.69648984, 0.41123343, ..., 0.2297639 , 0.34162075,
       0.58538676])

Подход 2: логистическая регресси

In [14]:
# Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! 
# Может пригодиться sklearn.preprocessing.StandartScaler.

import numpy as np
import pandas
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold,cross_val_score

features = pandas.read_csv('features.csv', index_col='match_id')
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
y = features['radiant_win'].to_frame()
features.drop(['duration',
               'radiant_win',
               'tower_status_radiant', 
               'tower_status_dire', 
               'barracks_status_radiant',
               'barracks_status_dire'], axis=1, inplace=True)
features_clean = features.fillna(0)
features_test_clean = features_test.fillna(0)

rescale_attr = StandardScaler()
features_fin = rescale_attr.fit_transform(features_clean)
Y=np.array(y)


kf = KFold(y.size, n_folds=5, shuffle=True, random_state=241)

In [19]:
#1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с 
# L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для 
# градиентного бустинга. 
from sklearn.linear_model import LogisticRegression
lr_qual =[]
def run_LogReg(features_fin,Y):
    for i in range(-5,5):
        C=10**i
        lr = LogisticRegression(C=C, random_state=241)
        start_time = datetime.datetime.now()
        lr_scores = cross_val_score(lr, features_fin, Y.ravel(), cv=kf, scoring='roc_auc')
        run_time=datetime.datetime.now() - start_time
        print ('Time elapsed:', run_time)
        print (lr_scores)
        lr_qual.append((i,C,run_time.total_seconds(),np.mean(lr_scores)))
    return lr_qual

In [20]:
lr_qual = run_LogReg(features_fin, Y)

Time elapsed: 0:00:03.029860
[0.69959206 0.69159358 0.69419291 0.69433159 0.69589176]
Time elapsed: 0:00:04.489936
[0.7161759  0.7083769  0.70960313 0.71017245 0.71192219]
Time elapsed: 0:00:08.621488
[0.72134945 0.71368677 0.71425051 0.71512064 0.71649386]
Time elapsed: 0:00:11.336046
[0.7216634  0.71377395 0.71440813 0.71540233 0.7164595 ]
Time elapsed: 0:00:11.218287
[0.72165841 0.7137031  0.71438941 0.71539496 0.71640454]
Time elapsed: 0:00:11.969031
[0.72165762 0.71369565 0.71438656 0.71539329 0.7163998 ]
Time elapsed: 0:00:12.130469
[0.72165702 0.7136947  0.71438623 0.71539423 0.71639952]
Time elapsed: 0:00:11.724836
[0.72165734 0.71369455 0.7143859  0.71539408 0.71639946]
Time elapsed: 0:00:11.818378
[0.72165734 0.71369457 0.71438593 0.71539409 0.71639938]
Time elapsed: 0:00:13.207464
[0.72165733 0.71369457 0.71438593 0.71539407 0.71639938]


In [21]:
# Подберите при этом лучший параметр регуляризации (C).
def bestC(lr_qual):
    lr_qual.sort(key=lambda x:x[3])
    return max(lr_qual, key=lambda x: x[3])[3], max(lr_qual, key=lambda x: x[3])[1]

In [23]:
# Какое наилучшее качество у вас получилось?
all_values = bestC(lr_qual)[0]
all_C = bestC(lr_qual)[1]
print ("best val=%.4f, with C=%.2f" % (all_values,all_C))
all_values

best val=0.7163, with C=0.01


0.716341462186996

In [24]:
# Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
time_required = max(lr_qual, key=lambda x: x[3])[2]
time_required #=11.336046
# GB@200trees t=323s

11.336046

In [32]:
#2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, 
#что вряд ли является хорошей идеей.
#Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, 
#d2_hero, ..., d5_hero. Уберите их из выборки,
features_clean_nocat=features_clean.copy()


In [33]:
def remove_cat_attr(features_nocat):
    del features_nocat['lobby_type']
    for n in range(1, 6):
        del features_nocat['r{}_hero'.format(n)]
        del features_nocat['d{}_hero'.format(n)]
    return features_nocat

In [34]:
#remove category-attributes
remove_cat_attr(features_clean_nocat)
features_clean_nocat[:2]

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0


In [35]:
#normalize attributes
features_fin_nocat = rescale_attr.fit_transform(features_clean_nocat)

In [36]:
# rerun crossval for dataset without category-attributes
lr_qual_nocat = run_LogReg(features_fin_nocat, Y)

Time elapsed: 0:00:02.696174
[0.69961316 0.69150564 0.69419646 0.69425458 0.69571482]
Time elapsed: 0:00:03.777219
[0.71630684 0.70828717 0.70971489 0.71020968 0.71172337]
Time elapsed: 0:00:07.729608
[0.72152794 0.71360976 0.71445497 0.71521698 0.71636831]
Time elapsed: 0:00:09.233237
[0.7218164  0.71370124 0.71462628 0.71551614 0.71634469]
Time elapsed: 0:00:09.620300
[0.72181205 0.7136378  0.7146095  0.71551012 0.71629945]
Time elapsed: 0:00:10.040947
[0.72181039 0.71363045 0.71460884 0.71551016 0.71629392]
Time elapsed: 0:00:10.249705
[0.72181043 0.71362965 0.71460893 0.71551033 0.71629306]
Time elapsed: 0:00:10.053217
[0.72181067 0.71362945 0.71460889 0.71551027 0.71629322]
Time elapsed: 0:00:10.065452
[0.72181072 0.71362952 0.71460893 0.71551026 0.71629323]
Time elapsed: 0:00:09.571548
[0.72181071 0.71362952 0.71460894 0.71551026 0.71629323]


In [37]:
# и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
nocat_values = bestC(lr_qual_nocat)[0]
nocat_C = bestC(lr_qual_nocat)[1]
print ("best val=%.6f, with C=%.2f with dropped category attributes" % (nocat_values,nocat_C))
nocat_values

best val=0.716401, with C=0.01 with dropped category attributes


0.7164009506527343

In [38]:
#Изменилось ли качество?
nocat_values/all_values

1.0000830448450613

In [40]:
#На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли 
# за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, 
# чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может 
# пригодиться фукнция unique или value_counts).
features_heros=features_clean.copy()
all_heroes1=[]
all_heroes2=[]
for n in range(1, 6):
    all_heroes1.append(features_heros['r{}_hero'.format(n)].unique())
    all_heroes2.append(features_heros['d{}_hero'.format(n)].unique())

all_names_of_heroes=[]
for i in range(0,5):
    for j in range(0,len(all_heroes2[i])):
        all_names_of_heroes.append(all_heroes2[i][j])
for i in range(0,5):
    for j in range(0,len(all_heroes1[i])):
        all_names_of_heroes.append(all_heroes1[i][j])

In [41]:
hero_names = list(set(all_names_of_heroes))
#hero_names
Num_heroes = len(hero_names)
print (Num_heroes)

108


In [48]:
#4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев.
#Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, 
#если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. 
#Ниже вы можете найти код, который выполняет данной преобразование. 
# N — количество различных героев в выборке
# Формируем "мешок слов" по героям
def bag_hero_to_num(X):
    X_pick = np.zeros((X.shape[0], Num_heroes))
    for i, match_id in enumerate(X.index):
        for p in range(1,6):
            X_pick[i, np.where(hero_names == X.ix[match_id, 'r%d_hero' % p])] = 1
            X_pick[i, np.where(hero_names == X.ix[match_id, 'd%d_hero' % p])] = -1
    return X_pick



In [49]:
# clean dataset with all attributes - to estimate value for new attribute based on hero-category 
X_pick = bag_hero_to_num(features_clean)
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [51]:
#Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
# copy cleaned + removed_cat-attr
features_clean_catToNum_base = features_clean_nocat.copy()
features_clean_catToNum_base.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [52]:
#add new-cat-digital to cleaned_removed_cat-attr
features_clean_catToNum = features_clean_catToNum_base.join(pandas.DataFrame(X_pick, index=features_clean_nocat.index))
features_clean_catToNum.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,98,99,100,101,102,103,104,105,106,107
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# normalize newly created dataset
features_fin_catToNum = rescale_attr.fit_transform(features_clean_catToNum)
#features_fin_catToNum[:2]

In [54]:
#4 .Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
lr_qual_corCat = run_LogReg(features_fin_catToNum, Y)

Time elapsed: 0:00:06.260029
[0.71907467 0.71117649 0.71512939 0.71288168 0.71586231]
Time elapsed: 0:00:08.847284
[0.74685672 0.74006714 0.74277168 0.74026225 0.74423779]
Time elapsed: 0:00:14.961524
[0.75515333 0.74931124 0.75181648 0.74901026 0.75305284]
Time elapsed: 0:00:20.309634
[0.75521596 0.74985583 0.75218088 0.74951097 0.75308879]
Time elapsed: 0:00:20.972533
[0.75513027 0.74989157 0.75213472 0.74953537 0.75293117]
Time elapsed: 0:00:21.668403
[0.75511969 0.74989604 0.75212742 0.74953884 0.75290574]
Time elapsed: 0:00:21.779120
[0.75511883 0.74989652 0.75212657 0.74953969 0.75290278]
Time elapsed: 0:00:24.243963
[0.75511879 0.74989678 0.75212649 0.74953983 0.75290249]
Time elapsed: 0:00:22.505003
[0.75511885 0.74989672 0.75212658 0.74953981 0.75290238]
Time elapsed: 0:00:21.136635
[0.75511887 0.74989671 0.75212658 0.74953978 0.75290239]


In [55]:
values_with_catToNum = bestC(lr_qual_corCat)[0]
C_with_catToNum = bestC(lr_qual_corCat)[1]
print ("best val=%.4f, with C=%.2f with correccly propagated category attributes" % (values_with_catToNum,C_with_catToNum)) 
values_with_catToNum

best val=0.7520, with C=0.01 with correccly propagated category attributes


0.7519704890317174

In [56]:
# ratio to previous version
values_with_catToNum/nocat_values

1.0496503226950977

In [59]:
# this is best model, make sense to optimise C again with coarse binning around "C_with_catToNum=0.01"
grid_C = np.linspace(0.001,0.1,20)
grid_C


array([0.001     , 0.00621053, 0.01142105, 0.01663158, 0.02184211,
       0.02705263, 0.03226316, 0.03747368, 0.04268421, 0.04789474,
       0.05310526, 0.05831579, 0.06352632, 0.06873684, 0.07394737,
       0.07915789, 0.08436842, 0.08957895, 0.09478947, 0.1       ])

In [60]:
lr_optimize_qual =[]
#features_fin_catToNum
for i in grid_C:
    C=i
    lrOpt = LogisticRegression(C=C, random_state=241)
    start_time = datetime.datetime.now()
    lrOpt_scores = cross_val_score(lrOpt, features_fin_catToNum, Y.ravel(), cv=kf, scoring='roc_auc')
    run_time=datetime.datetime.now() - start_time
    lr_optimize_qual.append((i,C,run_time.total_seconds(),np.mean(lrOpt_scores)))

In [61]:
lr_optimize_qual_values = bestC(lr_optimize_qual)[0]
lr_optimize_qual_C = bestC(lr_optimize_qual)[1]
print ("best val=%.6f, with C=%.5f with converted category attributes" % (lr_optimize_qual_values,lr_optimize_qual_C))
#best val=0.751984, with C=0.00621 with converted category attributes

best val=0.751984, with C=0.00621 with converted category attributes


In [62]:
#Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей
#(лучшей с точки зрения AUC-ROC на кросс-валидации). 
lrfin = LogisticRegression(C=lr_optimize_qual_C, random_state=241)
lrfin.fit(features_fin_catToNum, Y.ravel())

LogisticRegression(C=0.006210526315789474, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=241,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [63]:
# Prepate test dataset:
# 1. take test data with cleaned missing values
features_test_clean_nocat = features_test_clean.copy()

# 2. remove category attributes
remove_cat_attr(features_test_clean_nocat)

# 3. create attributes based on category - heroes type
X_pick_test = bag_hero_to_num(features_test_clean)

# 4. add newly created attributes to initial test matrix without categories
features_test_clean_catToNum = features_test_clean_nocat.join(pandas.DataFrame(X_pick_test, index=features_test_clean.index))

# 5. normalize attributes in final test dataset
features_test_fin1 = rescale_attr.fit_transform(features_test_clean_catToNum)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [64]:
rad_win_pred = lrfin.predict_proba(features_test_fin1)
rad_win_pred

array([[0.1786769 , 0.8213231 ],
       [0.24540716, 0.75459284],
       [0.81030389, 0.18969611],
       ...,
       [0.76469889, 0.23530111],
       [0.36947263, 0.63052737],
       [0.5690182 , 0.4309818 ]])

In [65]:
#Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).
print("min = %.3f, max= %.3f" % (rad_win_pred.min(), rad_win_pred.max()))



min = 0.004, max= 0.996


Report part 2:
Q1. Какое качество получилось у логистической регрессии над всеми исходными признаками? 1.2: Как оно соотносится с качеством градиентного бустинга? 1.3: Чем вы можете объяснить эту разницу? 1.4: Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

A1.1: 0.71634 (all_values в коде);
A1.2: качество лучше чем у очень глубокого дерева (0.71352 GB ntrees=200)
A1.3: Линейная регрессия оптимально проводит разделяющую гиперплоскость для объектов двух классов. Градиетный бустинг хуже подходит для разреженных данных
A1.4: время LR~11s (time_required в коду), у бустинга ~323s -> в 30 раз быстрее

Q2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? 2.2: Чем вы можете объяснить это изменение?

A2.1: Почти не меняется : ( в коде: значение values_with_catToNum=0.716401, отношение= nocat_values/all_values=1.0000830448450613)
A2.2: категориальные признаки использовались не верно в предыдущем шаге (как числовые), поэтому не вносили дополнительной полезной информации - просто шум. при удалении, получили примерно такое же качество (модель и в первом случае проигнорировала шум) 

Q3. Сколько различных идентификаторов героев существует в данной игре?
A3: 108

Q4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
A.4.1: 0.7520 (values_with_catToNum в коде) 
A.4.2: да, улучшилось на 5% (values_with_catToNum/nocat_values=1.04965032)
A.4.3: в данном случае категориальные признаки используются по назначению и несут дополнительную информацию, которая помогает в определении решения

Q5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
A.5.1: min = 0.004, max= 0.996